# Age, Gender and Emotion Detection

### Let's load our classfiers

In [1]:
from pathlib import Path
import cv2
import dlib
import sys
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from keras.models import load_model
from keras.preprocessing.image import img_to_array


#We use pretrained model VGG,(add keras Pre-trained Models) 
#We first load the model architecture and pre-trained weights
#We then add a task-specific classification layer on the top of the pre-trained model.
# There is a file containing the emotion values for all the data.

classifier = load_model('./model/emotion_little_vgg_2.h5')
pretrained_model = './Weights/weights.28-3.73.hdf5'

### Detecting Age, Gender, Emotion of people in any Image

In [2]:
from os import listdir
from os.path import isfile, join
import os
import cv2


# Define Image Path Here
image_path = "./images/"

emotion_classes = {0: 'Angry ' , 1: 'Fear', 2: 'Happy' , 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


#draw rectangle and put information text top,left of images

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_PLAIN,
               font_scale=1, thickness=1):
    
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    
    #It is the coordinates of the text string in the image.
    x, y = point
    
    #the rectangle above img and its position and colour.
    #It is the color of text string to be drawn. For BGR, we pass a tuple
    cv2.rectangle(image, (x+size[0], y), (x, y-size[1]), (0, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (100, 50, 255), thickness, lineType=cv2.LINE_AA)#the information text
    

# Define our model parameters
depth = 16
k = 8       #K:the widening factor, which multiplies the number of feature channels in each convolution layer.
weight_file = None
margin = 0.4    #for imdb
image_dir = None


# Get our weight file 
if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                              cache_dir=Path(sys.argv[0]).resolve().parent)

# load model and weights

img_size = 64  #default 64

model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)


#The get_frontal_face_detector() will return a detector that is a function we can use to retrieve the faces information.
#Each face is an object that contains the points where the image can be found.
detector = dlib.get_frontal_face_detector() # create the detector

#create a list (f for f in)
#method listdir() returns a list containing the names of the entries in the directory given by path
image_names = [f for f in listdir(image_path) if isfile(join(image_path, f))]


for image_name in image_names:
    frame = cv2.imread("./images/" + image_name) # read the image
    
    preprocessed_faces_emo = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Convert image into grayscale(opencv face detector expects gray images)
    
    
    #numpy.shape() function finds the shape of an array. By shape, we mean that it helps in finding the dimensions of an array.
    img_h, img_w, _ = np.shape(input_img)  #Return the shape of an array.
    
    
    # Use detector (dlib) to find face
    detected = detector(frame, 1) 
    
     # placeholder for cropped faces
    faces = np.empty((len(detected), img_size, img_size, 3))
    #len:Return the length (the number of items) of an object.
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):   #Return an enumerate object. iterable must be a sequence.
           
            x1 = d.left()       # left point
            y1 = d.top()        # top point
            x2 = d.right() + 1  # right point
            y2 = d.bottom() + 1 # bottom point
            w = d.width()       # width point
            h = d.height()      # height point
           
        #set all of the properties for the four margins 
        #add some margin to the face detected area to include a full head
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
           
            # Draw a rectangle(position, colour, thickness)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) # change to greyscale
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
           
           # Convert the field to a float
           # Regarding the division by 255, this is the maximum value of a byte 
           #(the input feature's type before the conversion to float32), so this will ensure 
           #that the input features are scaled between 0.0 and 1.0. 
            face_gray_emo = face_gray_emo.astype("float") / 255.0    
            
            #Keras provides the img_to_array() function for converting a loaded image in PIL format
            #into a NumPy array for use with deep learning models
            face_gray_emo = img_to_array(face_gray_emo)
            
            #Expand the shape of an array.
            #Insert a new axis that will appear at the axis position in the expanded array shape.
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

       
    
       # make a prediction for Age and Gender
        results = model.predict(np.array(faces)) # make a prediction
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1) # shape arr with 101 rows and 1 columns (It shapes an array without changing the data of array.)
        predicted_ages = results[1].dot(ages).flatten() #flatten:Return a copy of the array collapsed into one dimension

      
        # make a prediction for Emotion 
        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
            
      
    
      # draw results
        for i, d in enumerate(detected):
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                "Female" if predicted_genders[i][0] > 0.5 else "Male", emo_labels[i] )
           
        draw_label(frame, (d.left(), d.top()), label)

   

    cv2.imshow("A,G,E Detector", frame) # show the image
    filename = "output_images/"+image_name
    
    cv2.imwrite(filename,frame) # save result
    
    
    #we need to pause execution, as the window will be destroyed when the script stops, 
    #so we use cv2.waitKey to hold the window until a key is pressed, and after that,
    #we destroy the window and exit the script.
    
    cv2.waitKey(0)  # Wait for a key press to exit
    cv2.destroyAllWindows()   # Close all windows   

### Detection with webcam

In [3]:
from os import listdir
from os.path import isfile, join
import os
import cv2


# Define Image Path Here
#image_path = "./images/"

emotion_classes = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (0, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (100, 50, 255), thickness, lineType=cv2.LINE_AA)
    

# Define our model parameters
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = None

# Get our weight file 
if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                                cache_dir=Path(sys.argv[0]).resolve().parent)
# load model and weights
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)

detector = dlib.get_frontal_face_detector()



# Initialize Webcam (0 means the default video capture device in OS)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
  

    preprocessed_faces_emo = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
    
     # placeholder for cropped faces
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

        # make a prediction for Age and Gender
        results = model.predict(np.array(faces))
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()

        # make a prediction for Emotion 
        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
        
        # draw results
        for i, d in enumerate(detected):
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                        "Female" if predicted_genders[i][0] > 0.5 else "Male", emo_labels[i])
            draw_label(frame, (d.left(), d.top()), label)

    cv2.imshow("A,G,E Detector", frame)
    if cv2.waitKey(5) == 27:  # ESC key press
        break

cap.release()  # When everything is done, release the capture
cv2.destroyAllWindows()      